## Data Cleaning Summary
In this notebook, we focus on the initial cleaning and validation of the raw Steam dataset to prepare it for modeling and analysis. The dataset includes game metadata, pricing, reviews, achievements, and other attributes collected from [Steam Game Data on Kaggle](https://www.kaggle.com/datasets/artyomkruglov/gaming-profiles-2025-steam-playstation-xbox/). 

The hope is that the cleaned dataset serves as a reliable foundation for downstream modeling in Power BI. It should eliminate structural issues that could affect joins, aggregations, and filtering during Power BI reporting. The output of this notebook will be used as input for the normalization process that follows in the next notebook.



In [1]:
import pandas as pd
import numpy as np
import sys
import ast
import os
sys.path.append(os.path.abspath(".."))
from utils.data_utils import missing_value_summary

### Games

In [2]:
gamesdf = pd.read_csv("../data_steam/raw/games.csv")

In [3]:
games = gamesdf.copy()

In [4]:
gamesdf['gameid'].nunique()

98248

In [5]:
games.head(3)

,gameid,title,developers,publishers,genres,supported_languages,release_date
0,3281560,Horror Game To Play With Friends! Playtest,NaN,NaN,NaN,NaN,2024-10-21
1,3280930,Eternals' Path Playtest,NaN,NaN,NaN,NaN,2024-10-17
2,3280770,ANGST: A TALE OF SURVIVAL - Singleplayer Playtest,NaN,NaN,NaN,NaN,2024-10-13


In [6]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98248 entries, 0 to 98247
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   gameid               98248 non-null  int64 
 1   title                98245 non-null  object
 2   developers           92689 non-null  object
 3   publishers           92307 non-null  object
 4   genres               92699 non-null  object
 5   supported_languages  92742 non-null  object
 6   release_date         98248 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.2+ MB


There are 98248 unique games, and the number of rows in the table is 98248. So this means there are no duplicates - each row represents a unique game. 
The above shows a sample of the first three rows. As detailed by the dataset curator on Kaggle, the games table has the following columns:

* gameid - unique gameID on the Steam platform
* title - full title of the game
* developers - list of developers
* publishers - list of publishers
* genres - list of game genres
* supported_languages - languages available in the game as subtitles or voice-over
* release_date - game release date

There some columns with null values and we shall deal with these shortly. Notice that from the first three games the `title` contains the word 'Playtest'. We will be interested in variables like prices, genres, developers. Most 'playstests' don't include records for these, as shown below. Sowe will remove all 'playstest' games. From below, there are 5280 of them, about 5% of the dataset.

In [5]:
playtest_games = games[games['title'].astype(str).str.contains('Playtest')] 
playtest_num = len(playtest_games)
print(f'Number of playtest games: {playtest_num}\n')
playtest_perc = (playtest_num / len(games) ) * 100
print(f'Percentage of playtest games: {playtest_perc: .2f}%')
print("\n Number of NaNs per column:\n")
print(playtest_games.isna().sum())


Number of playtest games: 5280

Percentage of playtest games:  5.37%

 Number of NaNs per column:

gameid                    0
title                     0
developers             5273
publishers             5276
genres                 5273
supported_languages    5273
release_date              0
dtype: int64


In [8]:
# View them just to see if its the expected output.
playtest_games.head(5)

,gameid,title,developers,publishers,genres,supported_languages,release_date
0,3281560,Horror Game To Play With Friends! Playtest,NaN,NaN,NaN,NaN,2024-10-21
1,3280930,Eternals' Path Playtest,NaN,NaN,NaN,NaN,2024-10-17
2,3280770,ANGST: A TALE OF SURVIVAL - Singleplayer Playtest,NaN,NaN,NaN,NaN,2024-10-13
3,3279790,Montabi Playtest,NaN,NaN,NaN,NaN,2024-10-13
4,3278320,파이팅걸 유리 Playtest,NaN,NaN,NaN,NaN,2024-10-12


In [6]:
games = games[~games['title'].astype(str).str.contains('Playtest')] #remove Playtestgames

In [10]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 92968 entries, 5 to 98247
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   gameid               92968 non-null  int64 
 1   title                92965 non-null  object
 2   developers           92682 non-null  object
 3   publishers           92303 non-null  object
 4   genres               92692 non-null  object
 5   supported_languages  92735 non-null  object
 6   release_date         92968 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.7+ MB


#### Games - Missing values
Now exploring missing data in general after removing playstests. Table below indicates that the columns have <1% missing values. 

In [9]:
miss_per_col  = missing_value_summary(games)
miss_per_col

,Number of Missing Values,Percent of Total Values
publishers,662,0.71
developers,283,0.30
genres,274,0.29
supported_languages,231,0.25
gameid,0,0.00
title,0,0.00
release_date,0,0.00


Starting with `title`, since there are only 3 games with missing values, we shall remove these. This table has the game information, so there is no way we can find out the names of the games. From the output below, these also have no developers and publishers.

In [10]:
no_title_games = games[games['title'].isna()]
games = games[~games['title'].isna()]
no_title_games

,gameid,title,developers,publishers,genres,supported_languages,release_date


In [11]:
miss_per_col  = missing_value_summary(games)
miss_per_col

,Number of Missing Values,Percent of Total Values
publishers,662,0.71
developers,283,0.30
genres,274,0.29
supported_languages,231,0.25
gameid,0,0.00
title,0,0.00
release_date,0,0.00


For  developers, publishers, genres and supported_languages we shall place the null values with "None stated" to avoid losing more data.

In [12]:
miss_vals_col = list(miss_per_col[(miss_per_col['Number of Missing Values'] > 0)].index)
for col in miss_vals_col:
    games[col] = games[col].apply(lambda x: ['None stated'] if pd.isna(x) else x)


In [15]:
miss_per_col  = missing_value_summary(games)
miss_per_col

,Number of Missing Values,Percent of Total Values
gameid,0,0.0
title,0,0.0
developers,0,0.0
publishers,0,0.0
genres,0,0.0
supported_languages,0,0.0
release_date,0,0.0


In [560]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 92965 entries, 5 to 98247
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   gameid               92965 non-null  int64 
 1   title                92965 non-null  object
 2   developers           92965 non-null  object
 3   publishers           92965 non-null  object
 4   genres               92965 non-null  object
 5   supported_languages  92965 non-null  object
 6   release_date         92965 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.7+ MB


### Prices

In [13]:
pricesdf = pd.read_csv('../data_steam/raw/prices.csv')

In [14]:
prices = pricesdf.copy()

As per Kaggle doumentation, the prices table has the following columns:

* `gameid` - unique gameID on the Steam platform
* `usd` - game price in USD
* `eur` - game price in EUR
* `gbp` - game price in GBP
* `jpy` - game price in JPY
* `rub` - game price in RUB
* `date_acquired` - date of when the price information was recorded.
  
Based on the info below, the prices table has 4414273 rows. The table has prices in different currencies (`usd`, `eur` etc), on different dates (`date_acquired`) for the games. Notably each game's price can be 'acquired' on different days, hence there could be multiple entries for each game. The (original) `games` table had 98248 unique gameids, the `prices` table has 98465. So there are about 217 games with a price but no game information (in the original games table). We will use the original games table to weed out these games since we will have no information about them.

We will first check the 'playtest' games in the prices (using the `game_id`) to see if these have prices (they probably shouldn't) then remove these, so we have a leaner dataframe more in line with the cleaned games table.

In [20]:
prices.head(3)

,gameid,usd,eur,gbp,jpy,rub,date_acquired
0,3281560,NaN,NaN,NaN,NaN,NaN,2024-11-28
1,3280930,NaN,NaN,NaN,NaN,NaN,2024-11-28
2,3280770,NaN,NaN,NaN,NaN,NaN,2024-11-28


In [15]:
uniq_ids_prices = set(prices['gameid'])
uniq_ids_original_games = set(gamesdf['gameid']) #use the original games, before removing playtests
price_id_not_in_games = uniq_ids_prices - uniq_ids_original_games
print(f'Number of unique game ids in prices: {len(uniq_ids_prices)}')
print(f'Number of unique game ids in original games: {len(uniq_ids_original_games)}')
print(f'Number of unique game ids in prices but not in original games: {len(price_id_not_in_games)}')

Number of unique game ids in prices: 98465
Number of unique game ids in original games: 98248
Number of unique game ids in prices but not in original games: 217


Below, we see prices for which the game ids are not in the original games table and there are 217 such games. We remove these prices since we will not be able to get information for the corresponding game. 

In [16]:
gameid_not_in_games = prices[prices['gameid'].isin(list(price_id_not_in_games))]
prices = prices[prices['gameid'].isin(gamesdf['gameid'])] #remove these games
gameid_not_in_games.head()


,gameid,usd,eur,gbp,jpy,rub,date_acquired
361,1423450,9.99,9.75,8.50,1200.0,385.0,2024-11-28
1816,1378820,0.99,0.99,0.84,117.0,41.0,2024-11-28
2447,1360760,0.99,0.79,0.79,100.0,30.0,2024-11-28
2622,1355020,6.99,5.69,5.19,720.0,175.0,2024-11-28
2959,1344430,0.55,0.55,0.47,65.0,22.0,2024-11-28


In [17]:

#unmatched_playtest_ids = review_id_not_in_games.intersection(playtest_ids)
#print("Number of games in reviews not in games table but are in the playtest:", len(unmatched_playtest_ids))  
#remaining_unmatched = review_id_not_in_games - unmatched_playtest_ids
#print("Number of games in reviews that do not appear in games after removing playtests:", len(expected_remaining_unmatched))  


In [18]:
uniq_ids_prices = set(prices['gameid'])
uniq_ids_original_games = set(gamesdf['gameid']) #use the original games, before removing playtests
uniq_ids_updated_games = set(games['gameid'])
print(f'Number of unique game ids in new prices table: {len(uniq_ids_prices)}')
print(f'Number of unique game ids in original games: {len(uniq_ids_original_games)}')
print(f'Number of unique game ids in updated games: {len(uniq_ids_updated_games)}')

Number of unique game ids in new prices table: 98248
Number of unique game ids in original games: 98248
Number of unique game ids in updated games: 92965


Next we shall remove the playtest games. The attempt now is to get the prices table as close as possible to the updated `games` table. We will first check the 'playtest' games in the prices (using the `game_id`) to see if these have prices (they probably shouldn't) then remove these, so we have a leaner dataframe more in line with the cleaned games table. In the `prices` table, we shall remove the games that correspond to playtest games since these have no prices. Below we first confirm that indeed all the prices for the playtest games are NaNs, afterwhich we remove these.

In [19]:
playtest_ids = playtest_games['gameid']  # game ids with playtest in title in games df 
playtest_prices = prices[prices['gameid'].isin(playtest_ids)] #prices for games which are playtests
print(f'Number of rows in Playtests prices table: {playtest_prices.shape[0]}') #number of rows
print('Number of missing values per column:')
playtest_prices.isna().sum() #which of the playtest games are NaNs?

Number of rows in Playtests prices table: 233714
Number of missing values per column:


gameid                0
usd              233714
eur              233714
gbp              233714
jpy              233714
rub              233714
date_acquired         0
dtype: int64

In [20]:
prices = prices[~prices['gameid'].isin(playtest_ids)]
prices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4170948 entries, 5 to 4414272
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   gameid         int64  
 1   usd            float64
 2   eur            float64
 3   gbp            float64
 4   jpy            float64
 5   rub            float64
 6   date_acquired  object 
dtypes: float64(5), int64(1), object(1)
memory usage: 254.6+ MB


Below we see that the number of unique ids in the prices and games table are almost the same (with a difference of 3). I suspect these are the 3 games with no title that we removed earlier on, and this is confirmed below as well. We shall remove these gameids from prices. 

In [21]:
uniq_ids_prices = set(prices['gameid'])
uniq_ids_updated_games = set(games['gameid'])
print(f'Number of unique game ids in updated prices table: {len(uniq_ids_prices)}')
print(f'Number of unique game ids in updated games: {len(uniq_ids_updated_games)}')

Number of unique game ids in updated prices table: 92968
Number of unique game ids in updated games: 92965


In [22]:
#checking if the gameids in prices that dont appear in games are exactly the ones that have no title from before
set(prices['gameid']) - set(games['gameid'])  == set(no_title_games['gameid']) 

False

In [23]:
prices = prices[prices['gameid'].isin(games['gameid'])]
uniq_ids_prices = set(prices['gameid'])
uniq_ids_updated_games = set(games['gameid'])
print(f'Number of unique game ids in updated prices table: {len(uniq_ids_prices)}')
print(f'Number of unique game ids in updated games: {len(uniq_ids_updated_games)}')

Number of unique game ids in updated prices table: 92965
Number of unique game ids in updated games: 92965


In [24]:
set(games['gameid']) == set(prices['gameid']) #check if all IDs in games are in prices and vice versa

True

Just a reminder of how the prices table looks:

In [27]:
prices.head(3)

,gameid,usd,eur,gbp,jpy,rub,date_acquired
5,3278740,5.99,5.85,5.10,720.0,228.0,2024-11-28
10,3270850,3.99,3.99,3.89,470.0,165.0,2024-11-28
15,3267350,NaN,NaN,NaN,NaN,NaN,2024-11-28


We continue checking the NA's in prices. First we check which rows have an NaN in any column, i.e which game has no price in at least one currency.  The table below indicates theres quite a number of missing prices in the different currencies.

In [25]:
rows_with_nan = prices[prices.isnull().any(axis=1)]
#print('')
print('Sample of table with at least one missing price:\n')
print(rows_with_nan.head(3))
print('\nPercentage of missing values per column: ')
miss_prices = missing_value_summary(rows_with_nan)
miss_prices

Sample of table with at least one missing price:

     gameid   usd  eur   gbp    jpy    rub date_acquired
15  3267350   NaN  NaN   NaN    NaN    NaN    2024-11-28
17  3266470  3.49  NaN  3.00  406.0  140.0    2024-11-28
22  3263370  0.99  NaN  0.89  120.0   42.0    2024-11-28

Percentage of missing values per column: 


,Number of Missing Values,Percent of Total Values
eur,1463601,93.89
rub,767321,49.22
jpy,681478,43.72
gbp,669734,42.96
usd,668743,42.90
gameid,0,0.00
date_acquired,0,0.00


We are going to attempt to investigate the issue of the NaN prices in the prices table by extracting just one such game. As seen below, in the prices table this one game has no prices (NaN). Checking the game in the games table indicates that this game is 'Free to play' (listed under `genres`) so customers need not pay. Two ways to deal with this: 
* either remove this and risk losing a lot of data or
* replace this with 0 which may skew the price distribution if theres a lot of such games compared to those which have a price greater than 0. 

In [26]:
one_suspicious_game = prices[prices['gameid']==3267350]
one_suspicious_game.head()

,gameid,usd,eur,gbp,jpy,rub,date_acquired
15,3267350,NaN,NaN,NaN,NaN,NaN,2024-11-28
97912,3267350,NaN,NaN,NaN,NaN,NaN,2024-11-30
195685,3267350,NaN,NaN,NaN,NaN,NaN,2024-12-02
293331,3267350,NaN,NaN,NaN,NaN,NaN,2024-12-04
390988,3267350,NaN,NaN,NaN,NaN,NaN,2024-12-06


In [27]:
games[games['gameid']==3267350]

,gameid,title,developers,publishers,genres,supported_languages,release_date
15,3267350,Tiny Shooters,['madilumar'],['Thetinyverse'],"['Action', 'Free To Play']",['English'],2024-10-19


We shall extract all 'Free to play games' and study them further. The print out shows there are 8746 of such games, about 9% of the total.
We then check in the prices table if indeed all these games have NaN for all prices. 

In [28]:
free_to_play = games[games['genres'].astype(str).str.contains('Free To Play', case=False)] 
free_to_play.head(2)

,gameid,title,developers,publishers,genres,supported_languages,release_date
15,3267350,Tiny Shooters,['madilumar'],['Thetinyverse'],"['Action', 'Free To Play']",['English'],2024-10-19
28,3261860,CAIGE: Cum Overflow,['Zapt5454'],['Zapt5454'],"['Casual', 'Free To Play']",['English'],2024-10-18


In [29]:
num_free_to_play = len(free_to_play)
print(f'Number of free to play games: {num_free_to_play}')
free_to_play_perc = (len(free_to_play) / len(games) )*100
print(f'Percentage of free to play games: {free_to_play_perc: .2f}%')

Number of free to play games: 8750
Percentage of free to play games:  9.41%


The print out below shows that not all 'Free to play' games have a price of `NaN` in the prices table. This could suggest that the game was once 'free to play' and so the tag stuck with it. In any case, since the prices are available, we can assume that the game is no longer free to play. I am making the decision to remove this 'tag' from the `genres` for games that actually have a price in the `prices` table so that it accurately reflects the current state. We shall start with the usd column and see where the analysis takes us.

In [30]:
free_to_play_ids = free_to_play['gameid']
free_to_play_prices = prices[prices['gameid'].isin(free_to_play_ids)]

#missing_value_summary(free_to_play_prices)
#free_to_play_prices['gameid'].nunique()
#free_to_play_prices.info()
free_to_play_prices[~free_to_play_prices['usd'].isna()].head(2) #checking which of the supposed free to play games actually have prices.

,gameid,usd,eur,gbp,jpy,rub,date_acquired
340,1424420,24.99,24.99,20.99,2800.0,880.0,2024-11-28
1009,1402280,14.39,NaN,11.99,1600.0,520.0,2024-11-28


In [31]:
games[games['gameid']==1424420] #extracting one such game for further analysis

,gameid,title,developers,publishers,genres,supported_languages,release_date
340,1424420,Warchief,['Honikou Games'],['Honikou Games'],"['Action', 'Indie', 'Strategy', 'Free To Play'...","['English', 'French', 'Italian', 'German', 'Sp...",2024-06-27


In [32]:
games.loc[340, 'genres']

"['Action', 'Indie', 'Strategy', 'Free To Play', 'Early Access']"

So clearly below, one game classified as free to play has prices in all the currencies:

In [33]:
prices[prices['gameid']==1424420].head(3)

,gameid,usd,eur,gbp,jpy,rub,date_acquired
340,1424420,24.99,24.99,20.99,2800.0,880.0,2024-11-28
98238,1424420,24.99,24.99,20.99,2800.0,880.0,2024-11-30
196011,1424420,24.99,24.99,20.99,2800.0,880.0,2024-12-02


In [34]:
#the goal in this cell is just some clean up on the genres column so we can access the genres to remove the 'Free to play' tag.
#For instance, the genre is represented as a list inside a string: "['Action', 'Free To Play']" but we want it as a list ['Action', 'Free To Play']
#so we can loop through it
games['genres'] = games['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
games.head(3)

,gameid,title,developers,publishers,genres,supported_languages,release_date
5,3278740,NEURO,['Revolt Games'],['Strategy First'],[Action],"['English', 'Russian']",2024-10-11
10,3270850,Keep Your Eyes Open,['Texerikus'],['Texerikus'],[Indie],['English'],2024-10-21
15,3267350,Tiny Shooters,['madilumar'],['Thetinyverse'],"[Action, Free To Play]",['English'],2024-10-19


In [35]:
games_w_price = free_to_play_prices[~free_to_play_prices['usd'].isna()]#games with the free to play tag that have a usd price (not NaN) in the price table
target_ids = games_w_price['gameid']  

# Function to remove 'Free To Play' from the genre column for those with prices
def remove_free_to_play(genres):
    return [g for g in genres if g != 'Free To Play']
# Apply only to rows where id is in target_ids
games.loc[games['gameid'].isin(target_ids), 'genres'] = games.loc[games['gameid'].isin(target_ids), 'genres'].apply(remove_free_to_play)


In [36]:
#test
games.loc[340, 'genres']

['Action', 'Indie', 'Strategy', 'Early Access']

Now we shall check the 'Free to play' games again having removed the ones impersonating as free to play. We now expect all the records to indicate that there are null values for all currencies. The table shows that all the 'usd' entries (387528) are (correctly) null but the other currencies, 387483 are (correctly null). So about 45 (387528 - 387483) of the rows need further investigation. 

In [37]:
free_to_play = games[games['genres'].astype(str).str.contains('Free To Play')] 
num_free_to_play = len(free_to_play)
print(f'Number of free to play games: {num_free_to_play: }')
free_to_play_perc = (len(free_to_play) / len(games) )*100
print(f'Percentage of free to play games: {free_to_play_perc: .2f}%')
free_to_play_ids = free_to_play['gameid']
free_to_play_prices = prices[prices['gameid'].isin(free_to_play_ids)]
free_to_play_prices.isna().sum()


Number of free to play games:  8666
Percentage of free to play games:  9.32%


gameid                0
usd              387528
eur              387483
gbp              387483
jpy              387483
rub              387483
date_acquired         0
dtype: int64

Using just one currency to check which is not null. Interestingly, this is the same game. It has a NaN for 'usd' but for the other currencies it has a price. Perhaps it is 'free to play' in the USA only. In any case, since I will be using usd prices, and this is just one game, we shall remove it from the analysis.

In [38]:
suspicious_not_null_price = free_to_play_prices[~free_to_play_prices['gbp'].isna()]
suspicious_not_null_price

,gameid,usd,eur,gbp,jpy,rub,date_acquired
81147,382120,NaN,0.79,0.79,100.0,30.0,2024-11-28
178963,382120,NaN,0.79,0.79,100.0,30.0,2024-11-30
276596,382120,NaN,0.79,0.79,100.0,30.0,2024-12-02
374193,382120,NaN,0.79,0.79,100.0,30.0,2024-12-04
471791,382120,NaN,0.79,0.79,100.0,30.0,2024-12-06
569594,382120,NaN,0.79,0.79,100.0,30.0,2024-12-08
667527,382120,NaN,0.79,0.79,100.0,30.0,2024-12-10
765101,382120,NaN,0.79,0.79,100.0,30.0,2024-12-12
862693,382120,NaN,0.79,0.79,100.0,30.0,2024-12-14
960483,382120,NaN,0.79,0.79,100.0,30.0,2024-12-16


In [39]:
games = games[games['gameid']!=382120].copy()#remove the game 
prices = prices[prices['gameid']!=382120].copy()
free_to_play = games[games['genres'].astype(str).str.contains('Free To Play')] 
num_free_to_play = len(free_to_play)
print(f'Number of free to play games: {num_free_to_play: }')
free_to_play_perc = (len(free_to_play) / len(games) )*100
print(f'Percentage of free to play games: {free_to_play_perc: .2f}%')
free_to_play_ids = free_to_play['gameid']
free_to_play_prices = prices[prices['gameid'].isin(free_to_play_ids)]
free_to_play_prices.isna().sum()

Number of free to play games:  8665
Percentage of free to play games:  9.32%


gameid                0
usd              387483
eur              387483
gbp              387483
jpy              387483
rub              387483
date_acquired         0
dtype: int64

For analysis later, I will add a column that indicates if the game was free to play or not, then also in the prices, assign the price of 0. 

In [40]:
games['Free to play'] = 'No'
games.loc[games['gameid'].isin(free_to_play_ids), 'Free to play'] = 'Yes'
prices = prices.fillna(0.0)


In [45]:
games.head(3)

,gameid,title,developers,publishers,genres,supported_languages,release_date,Free to play
5,3278740,NEURO,['Revolt Games'],['Strategy First'],[Action],"['English', 'Russian']",2024-10-11,No
10,3270850,Keep Your Eyes Open,['Texerikus'],['Texerikus'],[Indie],['English'],2024-10-21,No
15,3267350,Tiny Shooters,['madilumar'],['Thetinyverse'],"[Action, Free To Play]",['English'],2024-10-19,Yes


In [46]:
prices.head(3)

,gameid,usd,eur,gbp,jpy,rub,date_acquired
5,3278740,5.99,5.85,5.10,720.0,228.0,2024-11-28
10,3270850,3.99,3.99,3.89,470.0,165.0,2024-11-28
15,3267350,0.00,0.00,0.00,0.0,0.0,2024-11-28


Now, we have removed the 'Free to Play' tag for games that actuakky have nonzero prices. Some games have prices of 0.00 but are missing the 'Free to play' tag. I think it is safe to assign these games this tag just for un-ambiguois analysis later.  We do that in the next cell.

In [41]:
games['genres'] = games['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


price_lookup = prices.set_index('gameid')['usd'].to_dict()

def add_free_to_play_genre(gameid, genres):
    price = price_lookup.get(gameid, None)
    if price == 0.0 and "Free to Play" not in genres:
        return genres + ["Free to Play"]
    return genres

games['genres'] = games.apply(lambda row: add_free_to_play_genre(row['gameid'], row['genres']), axis=1)

In [42]:
games[games['gameid']==1330080]

,gameid,title,developers,publishers,genres,supported_languages,release_date,Free to play
3433,1330080,The Fantastic Adventure of Monsieur Grape,['Patrick Murrell'],['Patrick Murrell'],"[Adventure, Free to Play]",['English'],2020-07-03,No


In [43]:
games['Free to play'] = games['genres'].apply(
    lambda genres: "Yes" if "Free to Play" in genres else "No"
)


In [44]:
games[games['gameid']==1330080]

,gameid,title,developers,publishers,genres,supported_languages,release_date,Free to play
3433,1330080,The Fantastic Adventure of Monsieur Grape,['Patrick Murrell'],['Patrick Murrell'],"[Adventure, Free to Play]",['English'],2020-07-03,Yes


Steam uses regional pricing so with the missing values in other currencies it is not a simple matter of converting the price from one currency to another. Furthermore, since we have prices recorded over time, there is the issue of the varying exchange rate to consider. Therefore, instead, as already alluded to earlier on,for our analysis we shall focus only on USD prices. For our table to use less space, we remove the other currencies. Perhaps later on we could try to include them in the analysis. 

In [47]:
cols_to_drop = ['eur','gbp','jpy','rub']
prices = prices.drop(columns= cols_to_drop)


## Reviews table

We now look at the table with reviews. As per the documentation, this table has the following columns: 
* `reviewid` - unique reviewID (Serial Key)
* `playerid` - userID on the Steam platform who submitted the review
* `gameid` - gameID for which the review was written
* `review` - user-submitted review for the game
* `helpful` - number of users who found the review helpful
* `funny` - number of users who found the review funny
* `awards` - number of awards given to the review
* `posted `- timestamp of when the review was posted

Our main focus is the `gameid` since, later on, this can help in finding the number of reviews per game. We will drop the `review` column since it will not be useful in our analyses. 

In [49]:
reviewsdf = pd.read_csv('../data_steam/raw/reviews.csv')

In [50]:
reviews = reviewsdf.copy()
reviews.head(3)

,reviewid,playerid,gameid,review,helpful,funny,awards,posted
0,639543,76561198796340888,730,Goud gamę i have 3 vac ban acont but i stilll ...,0,0,0,2018-03-22
1,639544,76561198028706627,393380,---{ Graphics }---☐ You forget what reality is...,0,0,0,2025-01-03
2,639545,76561198028706627,10,One of the best FPS games :),0,0,0,2012-05-13


In [51]:
reviews = reviews.drop(columns=['review'])
reviews.head(2)

,reviewid,playerid,gameid,helpful,funny,awards,posted
0,639543,76561198796340888,730,0,0,0,2018-03-22
1,639544,76561198028706627,393380,0,0,0,2025-01-03


We will want to check if all the game ids in the reviews table appear in our main games table. It would be problematic to have a review for a game but no way to verify which game it is. We first check the orginal games table, before removing any data.

In [52]:
original_unmatched_ids = set(reviewsdf['gameid']) - set(gamesdf['gameid'])
print(f"Total review gameids missing in original games: {len(original_unmatched_ids)}")


Total review gameids missing in original games: 161


In [53]:
num_games_reviewed = set(reviews['gameid'])
tot_games = set(games['gameid'])
print(f'Number of games reviewed is: {len(num_games_reviewed)}')
print(f'Total number of games is: {len(tot_games)}')

Number of games reviewed is: 51910
Total number of games is: 92964


From above it appears not all games were reviewed (so some games have zero reviews). With this in mind, we can create a variable "HasReview", perhaps it will be important later. But first we will check if all `gameid`s in `reviews` appear in `games`. It appears that in the `reviews` table, there are some `gameid`s that do not appear in the main games table (235 of these). This is concerning as the `games` table is the main games table with all game related information. Perhaps these rogue games are those we removed that were playtests. 

In [54]:
#set(reviews['gameid'])
#set(reviews['gameid']) == set(games['gameid'])
review_id_not_in_games = list(set(reviews['gameid']) - set(games['gameid'])) #IDs in reviews not in games

print(f'Number of games in reviews table that do not appear in games table: {len(review_id_not_in_games)}')

Number of games in reviews table that do not appear in games table: 235


In [55]:
reviews_ids = set(reviews['gameid'])
games_ids = set(games['gameid'])
# IDs in reviews that don't exist in games
review_id_not_in_games = reviews_ids - games_ids
print("Number of games in reviews table that do not appear in games table:", len(review_id_not_in_games))  
unmatched_playtest_ids = review_id_not_in_games.intersection(playtest_ids)
print("Number of games in reviews not in games table but are in the playtest:", len(unmatched_playtest_ids))  
remaining_unmatched = review_id_not_in_games - unmatched_playtest_ids
print("Number of games in reviews that do not appear in games after removing playtests:", len(remaining_unmatched))  


Number of games in reviews table that do not appear in games table: 235
Number of games in reviews not in games table but are in the playtest: 72
Number of games in reviews that do not appear in games after removing playtests: 163


We have 72 playtest games. We shall remove these from the reviews for consistency. Then we continue to investigate which IDs are not in main games tables.

In [56]:
reviews = reviews[~reviews['gameid'].isin(playtest_ids)]
# sanity check
games_ids = set(games['gameid'])
reviews_ids = set(reviews['gameid'])# Gameids in reviews (after playtest removal)
unmatched_review_ids = reviews_ids - games_ids # IDs in reviews that are NOT in games

print(f"Unmatched review gameids (post-playtest-removal): {len(unmatched_review_ids)}")  # Expect 163


Unmatched review gameids (post-playtest-removal): 163


We will also remove the no title games like we did for games and prices (in this case its just one of the games appearing in reviews)


In [57]:
no_title_games['gameid'].isin(reviews['gameid'])

2853     False
39705    False
77700     True
Name: gameid, dtype: bool

In [58]:
reviews = reviews[~reviews['gameid'].isin(no_title_games['gameid'])]

In [59]:
# sanity check
games_ids = set(games['gameid'])
reviews_ids = set(reviews['gameid'])# Gameids in reviews (after playtest and no title games removal)
unmatched_review_ids = reviews_ids - games_ids # IDs in reviews that are NOT in games

print(f"Unmatched review gameids (post-playtest-no-title-removal): {len(unmatched_review_ids)}")  # Expect 162


Unmatched review gameids (post-playtest-no-title-removal): 162


We now have 162 games that appear in reviews but do not appear in games. Since these games have no information associated with them in the games table, I am making the decision to remove these. Perhaps it is a case of missing data in games table (perhaps de-listed games) or incorrectly captured ids in the review table. In any case, with no way to verify game information (like title, developers etc) these would be of little use, hence the decision. 

In [60]:
# Keep only reviews whose gameid appears in the main games table
reviews = reviews[reviews['gameid'].isin(games['gameid'])].copy()


## Achievements 
The table achievements contains:
* `achievementid` - unique achievementID, constructed as gameID + '_' + achievementNotUniqueID
* `gameid` - unique gameID on the Steam platform
* `title` - achievement title
* `description` - description of how to unlock the achievement

For data consistency here, we will only keep the achievements that have `gameid`s in the main games table. We will also drop the `title` and `description` for our analyses.

In [61]:
achievementsdf = pd.read_csv('../data_steam/raw/achievements.csv')

In [62]:
achievements = achievementsdf.copy()

In [63]:
achievements.head()

,achievementid,gameid,title,description
0,2621440_ACH_FIRST_KILL,2621440,FIRST KILL,You should kill ONE enemy.
1,2621440_ACH_0_LEVEL_COMPLETED,2621440,TUTORIAL COMPLETED,You should complete tutorial.
2,2621440_ACH_1_LEVEL_COMPLETED,2621440,FIRST LEVEL,You should complete first level
3,2621440_ACH_2_LEVEL_COMPLETED,2621440,SECOND LEVEL,You should complete second level
4,2621440_ACH_3_LEVEL_COMPLETED,2621440,THIRD LEVEL,You should complete third level


In [64]:
unmatched_achievements = achievements[~achievements['gameid'].isin(games['gameid'])].copy() #in case I want to come back later and analyse
achievements = achievements[achievements['gameid'].isin(games['gameid'])].copy()
achievements = achievements.drop(columns= ['title', 'description'])

In [65]:
achievements.head(2)

,achievementid,gameid
0,2621440_ACH_FIRST_KILL,2621440
1,2621440_ACH_0_LEVEL_COMPLETED,2621440


In [62]:
unmatched_achievements['gameid'].nunique()

486

In [66]:
unmatched_achievements.head(3)

,achievementid,gameid,title,description
9761,2884310_AC_OpenTheDoor,2884310,Open the door,Replace the Bypass chip of the door control.
9762,2884310_AC_NANDDone,2884310,NAND Designer,Create a NAND gate.
9763,2884310_AC_Chapter1Done,2884310,Ghost in the Grid,Complete the Chapter 1.


## History 
The table history contains:
* `playerid` - unique userID on the Steam platform who earned achievementID
* `achievementid` - unique achievementID, constructed as gameID + '_' + achievementNotUniqueID
* `date_acquired` - timestamp of when the achievement was earned

For data consistency here, I will keep only the rows where the achievementid is in the cleaned achievements table.

In [67]:
historydf = pd.read_csv("../data_steam/raw/history.csv")

In [68]:
history = historydf.copy()

In [69]:
history.head(3)

,playerid,achievementid,date_acquired
0,76561198220441373,403640_ACH_1,2019-12-18 15:33:43
1,76561198220441373,403640_ACH_2,2019-12-18 23:49:51
2,76561198220441373,403640_ACH_3,2019-12-19 23:05:07


In [70]:
history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10693879 entries, 0 to 10693878
Data columns (total 3 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   playerid       int64 
 1   achievementid  object
 2   date_acquired  object
dtypes: int64(1), object(2)
memory usage: 244.8+ MB


Luckily for this table, it appears there aren't any missing values. 

In [71]:
miss_per_col  = missing_value_summary(history)
miss_per_col

,Number of Missing Values,Percent of Total Values
playerid,0,0.0
achievementid,0,0.0
date_acquired,0,0.0


There are 1928259 unique achievements and from below, it appears that quite a number of achievements (1043018) haven't been 'achieved'. 

In [72]:
# get sets of unique achievement IDs from both tables
achievement_ids_master = set(achievements['achievementid'])
achievement_ids_history = set(history['achievementid'])

unearned_achievements = achievement_ids_master - achievement_ids_history #achievements that have never been earned by any player

orphan_achievements = achievement_ids_history - achievement_ids_master #achievements in history that are not listed in the master achievements table 

perc_unearned = (len(unearned_achievements) / len(achievement_ids_master)) * 100 # percentage of achievements that have never been earned

print(f" Total unique achievements in achievements table: {len(achievement_ids_master)}")
print(f" Achievements never earned by any player: {len(unearned_achievements)}")
print(f" Percentage of unearned achievements: {perc_unearned:0.2f}%")
print(f" Achievements in history but missing in achievements table: {len(orphan_achievements)}")


print("\nSample unearned achievement IDs:")
print(list(unearned_achievements)[:5])


 Total unique achievements in achievements table: 1928259
 Achievements never earned by any player: 1043018
 Percentage of unearned achievements: 54.09%
 Achievements in history but missing in achievements table: 1747

Sample unearned achievement IDs:
['1732050_BANDIT_ACHIEVEMENT', '733500_582544', '1886830_ACHV@Speedrun1', '3169220_Dichotomy_07', '672150_LESVT10']


From below, it appears all the `achievementid`s in history but not in main `achievements` table are those in the `unmatched_achievements` which in turn correspond to `achievementid`s for games not in our main `games` table.

In [86]:
unmatched_ids = set(unmatched_achievements['achievementid'])
# How many orphan achievements are in unmatched_achievements
orphans_in_unmatched = orphan_achievements.intersection(unmatched_ids)
print(f"Number of orphan achievements found in unmatched_achievements: {len(orphans_in_unmatched)}")

# Check if all orphan achievements are present
all_orphans_present = orphan_achievements.issubset(unmatched_ids)
print(f"Are all orphan achievements present in unmatched_achievements? {all_orphans_present}")


Number of orphan achievements found in unmatched_achievements: 1747
Are all orphan achievements present in unmatched_achievements? True


In [73]:
history_clean = history[history['achievementid'].isin(achievements['achievementid'])].copy()

### Players
According to the documentation, the Players table contains the following:
* `playerid` - unique userID on the Steam platform
* `country` - the country in which the user resides
* `created` - date of creation of the gaming profile


In [74]:
playersdf = pd.read_csv("../data_steam/raw/players.csv")

In [75]:
players = playersdf.copy()

In [76]:
players.head()

,playerid,country,created
0,76561198287452552,Brazil,2016-03-02 06:14:20
1,76561198040436563,Israel,2011-04-10 17:10:06
2,76561198049686270,NaN,2011-09-28 21:43:59
3,76561198155814250,Kazakhstan,2014-09-24 19:52:47
4,76561198119605821,NaN,2013-12-26 00:25:50


There are 424683 unique player ids (hard to say if these are unique players since a person can create multiple accounts). The `country` column has 42% missing values. We wil replace these with `Unknown` just to avoid losing data. The `created' column also has some missing values which we cannot reliably impute. So we will leave these in for now and deal with them in the normalisation section.

In [94]:
players['playerid'].nunique()

424683

In [95]:
miss_per_col  = missing_value_summary(players)
miss_per_col

,Number of Missing Values,Percent of Total Values
country,177868,41.88
created,47669,11.22
playerid,0,0.00


In [77]:
players['country'] = players['country'].fillna('Unknown')
miss_per_col  = missing_value_summary(players)
miss_per_col

,Number of Missing Values,Percent of Total Values
created,47669,11.22
playerid,0,0.00
country,0,0.00


### Saving dataframes

In [78]:
prices.to_csv('../data_steam/cleaned/prices_cleaned.csv', index=False)
games.to_csv('../data_steam/cleaned/games_cleaned.csv', index=False)
achievements.to_csv('../data_steam/cleaned/achievements_cleaned.csv', index=False)
reviews.to_csv('../data_steam/cleaned/reviews_cleaned.csv', index=False)
players.to_csv('../data_steam/cleaned/players_cleaned.csv', index=False)
history_clean.to_csv('../data_steam/cleaned/history_cleaned.csv', index=False)

## Summary
This is what we have done in this notebok:
* Removed irrelevant or inconsistent records, such as playtest games

* Handled missing values appropriately across all key fields, with one exception

* Converted stringified lists (e.g., developers, genres) into actual Python lists for later normalisation

* Ensured data consistency across tables (e.g., all references to gameid are valid and aligned)

Next step is performing data normalisation to get the tables ready for Power BI.

In [79]:
prices.describe()

,gameid,usd
count,4.170768e+06,4.170768e+06
mean,1.564553e+06,6.834381e+00
std,8.429518e+05,1.328067e+01
min,1.000000e+01,0.000000e+00
25%,8.528800e+05,9.900000e-01
50%,1.500220e+06,3.990000e+00
75%,2.243900e+06,9.990000e+00
max,3.418820e+06,9.999800e+02


In [98]:
high_prices = prices[prices['usd']>300]

In [99]:
high_prices

,gameid,usd,date_acquired
30185,3013840,325.00,2024-11-28
59129,2504210,999.98,2024-11-28
59238,2499620,999.98,2024-11-28
76546,1200520,999.00,2024-11-28
127873,3013840,325.00,2024-11-30
...,...,...,...
4293938,1200520,999.00,2025-02-22
4345916,3013840,500.00,2025-02-24
4374901,2504210,999.98,2025-02-24
4375010,2499620,999.98,2025-02-24


In [100]:
games[games['gameid'].isin(high_prices['gameid'])]

,gameid,title,developers,publishers,genres,supported_languages,release_date,Free to play
30006,3013840,True Love,['A Guy'],['Whoes heart broken'],"[Action, Adventure, Casual, Indie]",['English'],2024-06-27,No
58974,2504210,The Leverage Game Business Edition,['A&S Inc.'],['A&S Inc.'],"[Indie, Simulation]","['English', 'Japanese']",2023-08-26,No
59083,2499620,The Leverage Game,['A&S Inc.'],['A&S Inc.'],"[Indie, Simulation]","['English', 'Japanese']",2023-08-25,No
76342,1200520,Ascent Free-Roaming VR Experience,['Fury Games'],['Fury Games'],[Action],['English'],2019-12-27,No
